# IMPORT DES LIBRAIRIES

In [1]:
import pandas as pd
import numpy as np

# OPTIONS PANDAS 

In [2]:
pd.options.display.float_format = '{:.0f}'.format

# OUVERTURE DE NOTRE DATASET

In [3]:
from DataPreprocessor import DataPreprocessor
import csv

In [4]:
with open('../DATA/IN/new_extract_gold_dvf.csv', 'r', encoding='utf-8') as file:
    csv_reader = csv.reader(file, delimiter=';')

    next(csv_reader)  # Skip the header row
    data = list(csv_reader)

data =  np.copy(data)

In [5]:
preprocessor = DataPreprocessor.DataPreprocessor()

In [6]:
X = preprocessor._fill_na(data)

In [7]:
x_filtered = preprocessor._handle_outliers(X)

In [8]:
x_filtered

array([['174619', '177000.0', '2021-01-08', ..., '1', '33320',
        'Appartement'],
       ['174620', '153000.0', '2021-01-08', ..., '1', '33800',
        'Appartement'],
       ['174621', '223170.0', '2021-01-08', ..., '1', '33127',
        'Appartement'],
       ...,
       ['349230', '155000.0', '2023-05-02', ..., '0', '33830', 'Maison'],
       ['349231', '111150.0', '2023-06-28', ..., '0', '33210', 'Maison'],
       ['349232', '1072000.0', '2023-05-05', ..., '0', '33470', 'Maison']],
      dtype='<U11')

In [9]:
x_filtered = preprocessor._prepare_final_dataset(x_filtered)

In [10]:
x_filtered

array([['174619', '177000.0', '0.0', ..., '1', '33320', 'Appartement'],
       ['174620', '153000.0', '0.0', ..., '1', '33800', 'Appartement'],
       ['174621', '223170.0', '0.0', ..., '1', '33127', 'Appartement'],
       ...,
       ['349230', '155000.0', '600.0', ..., '0', '33830', 'Maison'],
       ['349231', '111150.0', '51.0', ..., '0', '33210', 'Maison'],
       ['349232', '1072000.0', '0.0', ..., '0', '33470', 'Maison']],
      dtype='<U11')

In [11]:
from BaselineFiter import BaselineFiter

fiter = BaselineFiter.BaselineFiter()

In [12]:
fiter.fit(x_filtered)

MemoryError: Unable to allocate 68.7 GiB for an array with shape (123591, 4662) and data type <U32